<a href="https://colab.research.google.com/github/oj-o/Art-Tech-Science/blob/main/Sensor_OSC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import serial
from pythonosc import udp_client
import time

# --- Arduino Setup (Conceptual - not run in Colab) ---
# On your Arduino, you would:
# 1. Connect your sensor to an analog or digital pin.
# 2. Write code to read the sensor value.
# 3. Print the sensor value to the Serial monitor (e.g., `Serial.println(sensorValue);`).
#    Make sure to include a delimiter if sending multiple values (e.g., comma).
#    Include a small delay (`delay()`) to avoid flooding the serial port.

# --- Colab Setup ---
# 1. Identify your Arduino's serial port name. This varies depending on your OS.
#    Common names:
#    Windows: 'COMx' (e.g., 'COM3')
#    macOS: '/dev/tty.usbmodemXXXX' or '/dev/tty.wchusbserialXXXX'
#    Linux: '/dev/ttyACM0' or '/dev/ttyUSB0'
#    You might need to try a few or check your system's device manager.
#    In Colab, you'll likely need to use a library like `pyserial` and potentially
#    forward the serial port from your local machine, which can be complex.
#    A simpler approach for demonstration in Colab is to simulate the serial data.
#    For a real application, you would run this Python script on your local machine
#    connected to the Arduino.

# --- Configuration ---
# Replace with your Arduino's serial port
# In Colab, we'll simulate this
# serial_port = '/dev/ttyACM0'
# baud_rate = 9600 # Must match your Arduino's Serial.begin()

# OSC Destination
osc_ip = "127.0.0.1" # IP address of the OSC server (e.g., Max/MSP, Pure Data)
osc_port = 5005      # Port the OSC server is listening on

# --- Simulation ---
# Simulate reading from a serial port for demonstration in Colab
simulate_serial_data = True
simulated_value = 500 # Starting simulated value
simulated_increment = 10
max_simulated_value = 1023 # Typical analog read range
min_simulated_value = 0

# --- Main Code ---

# Create an OSC client
client = udp_client.SimpleUDPClient(osc_ip, osc_port)

# Attempt to open the serial port (will fail in Colab unless port forwarding is set up)
# In a real local environment, you would use:
# try:
#     ser = serial.Serial(serial_port, baud_rate, timeout=1)
#     time.sleep(2) # Give the serial port time to initialize
#     print(f"Serial port {serial_port} opened successfully.")
# except serial.SerialException as e:
#     print(f"Error opening serial port {serial_port}: {e}")
#     print("Please check the port name and that the Arduino is connected.")
#     # Exit or handle the error appropriately
#     ser = None # Set ser to None if opening fails

print(f"Sending OSC messages to {osc_ip}:{osc_port}")

# --- Main Loop ---
try:
    # if ser: # Only run if serial port was opened successfully
    while True:
        if simulate_serial_data:
            # Simulate reading data
            sensor_value_str = str(simulated_value)
            print(f"Simulating read: {sensor_value_str}")

            # Update simulated value
            simulated_value += simulated_increment
            if simulated_value > max_simulated_value or simulated_value < min_simulated_value:
                simulated_increment *= -1 # Reverse direction
                simulated_value += simulated_increment # Step back to stay within bounds

            # Simulate a delay
            time.sleep(0.1) # Simulate read rate

        # else: # Real serial read (would not work in Colab without setup)
            # Read a line from the serial port
            # line = ser.readline().decode('utf-8').strip()
            # if line:
            #     print(f"Read from serial: {line}")
            #     sensor_value_str = line
            # else:
            #     # No data received, continue the loop
            #     continue

        try:
            # Convert the string data to a number (integer or float)
            # Adjust based on what your Arduino is sending
            sensor_value = int(sensor_value_str)

            # --- Send OSC Message ---
            # Define the OSC address (e.g., '/sensor/value', '/arduino/data')
            osc_address = "/sensor/value"

            # Send the value as an OSC message
            client.send_message(osc_address, sensor_value)
            print(f"Sent OSC message: {osc_address}, value: {sensor_value}")

        except ValueError:
            print(f"Could not convert received data '{sensor_value_str}' to number.")
        except Exception as e:
            print(f"An error occurred during OSC sending: {e}")

except KeyboardInterrupt:
    print("Stopping script.")
# finally:
    # if ser and ser.isOpen():
        # ser.close()
        # print("Serial port closed.")

